In [42]:
import scala.util.matching
import org.apache.spark.rdd.RDD
import spark.implicits._ 

println("all good")

Unknown Error: <console>:31: error: stable identifier required, but this.$line7$read.spark.implicits found.
       import spark.implicits._
                    ^


data is in the format:

{
"conversionSymbol": "",
"volumeto": 6685400.72, 
"high": 9165.89, 
"low": 9144.6, 
"time": 1595030400, 
"volumefrom": 730.39, 
"close": 9151.68, 
"open": 9156.79, 
"conversionType": "direct"
}


In [91]:
case class TradingInfo (conversionSymbol: String, 
                        volumeto: Double, 
                        high: Double, 
                        low: Double, 
                        time: Int,
                        volumefrom: Double,
                        close: Double,
                        open: Double,
                        conversionType: String
                       )

def createTradingInfo(data: String): TradingInfo = {
    val s = data.split(", ")
    return TradingInfo(s(0).split(": ")(1), s(1).split(": ")(1).toDouble, s(2).split(": ")(1).toDouble, 
                       s(3).split(": ")(1).toDouble, s(4).split(": ")(1).toInt, s(5).split(": ")(1).toDouble, 
                       s(6).split(": ")(1).toDouble, s(7).split(": ")(1).toDouble, s(8).split(": ")(1))
}

val filename = "../Datasets/day_json.txt"
//val johnFileName = "/home/john/ID2221-Labs/project/Datasets"
val tradingInformation = sc.textFile(filename).
            flatMap(l => l.split("},")).
            //map(x => x.replace("\\", "")).
            map(x => x.replace("[", "")).
            map(x => x.replace("{", "")).
            map(x => x.replace("\"", "")).
            map(createTradingInfo)
            .cache()

tradingInformation.take(5).foreach(println)

println("All good")

TradingInfo(\\,3452435.9,9181.73,9167.58,1595120400,376.32,9168.16,9181.73,\direct\)
TradingInfo(\\,3583760.67,9175.47,9162.64,1595124000,390.9,9165.67,9168.16,\direct\)
TradingInfo(\\,4264728.56,9165.67,9146.8,1595127600,465.73,9159.37,9165.67,\direct\)
TradingInfo(\\,2737686.63,9163.84,9153.66,1595131200,298.82,9157.3,9159.37,\direct\)
TradingInfo(\\,3526995.72,9158.03,9149.38,1595134800,385.44,9149.81,9157.3,\direct\)
All good


defined class TradingInfo
filename = ../Datasets/day_json.txt
tradingInformation = MapPartitionsRDD[158] at map at <console>:47


createTradingInfo: (data: String)TradingInfo


MapPartitionsRDD[158] at map at <console>:47

In [216]:
// Data Action


// val df = tradingInformation.toDF("conversionSymbol","volumeto", "high", "low", "time", 
//                                  "volumefrom", "close","open","conversionType").show(5)


import org.apache.spark.sql.functions.{min, max, mean, stddev, typedLit}
import org.apache.spark.sql.types.{IntegerType, DoubleType}

val df = spark.createDataFrame(tradingInformation).sort("time")
val df_1 = df.withColumn("midPrice",($"high" + $"low")/2)

val statistics = df_1.agg(mean("midPrice"), 
                         stddev("midPrice"), 
                         min("midPrice"), 
                         max("midPrice")).head()

val midPrice_mean = statistics.getDouble(0)
val midPrice_std = statistics.getDouble(1)
val midPrice_min = statistics.getDouble(2)
val midPrice_max = statistics.getDouble(3)
val midPrice_range = midPrice_max - midPrice_min
println(midPrice_range)

// from -1 to 1 
//val df_2 = df_1.withColumn("midPriceNorm",($"midPrice" - midPrice_mean)/midPrice_std)
// from 0 to 1 
val df_2 = df_1.withColumn("midPriceNorm",(($"midPrice" - midPrice_min)/midPrice_range).cast(DoubleType)).cache()

df_2.show(5)
df_2.printSchema()

println("All good")

7885.93
+----------------+-------------+-------+-------+----------+----------+-------+-------+--------------+--------+-------------------+
|conversionSymbol|     volumeto|   high|    low|      time|volumefrom|  close|   open|conversionType|midPrice|       midPriceNorm|
+----------------+-------------+-------+-------+----------+----------+-------+-------+--------------+--------+-------------------+
|              \\|8.132674979E7|8470.44|8261.87|1570629600|   9681.19|8460.44|8261.87|      \direct\|8366.155|0.48730967685485427|
|              \\|8.241513365E7|8676.36| 8459.8|1570633200|   9628.14|8580.84|8460.44|      \direct\| 8568.08| 0.5129154075676553|
|              \\|3.312488617E7|8652.44|8533.37|1570636800|    3853.6|8590.17|8580.84|      \direct\|8592.905| 0.5160634192796539|
|              \\|1.879112661E7|8658.72|8590.17|1570640400|   2180.23|8650.17|8590.17|      \direct\|8624.445| 0.5200629475534274|
|              \\|2.599451864E7|8697.56|8612.02|1570644000|   3005.19|8621.

df = [conversionSymbol: string, volumeto: double ... 7 more fields]
df_1 = [conversionSymbol: string, volumeto: double ... 8 more fields]
statistics = [9022.352339736673,1533.4194619599898,4523.264999999999,12409.195]
midPrice_mean = 9022.352339736673
midPrice_std = 1533.4194619599898
midPrice_min = 4523.264999999999
midPrice_max = 12409.195
midPrice_range = 7885.93
df_2 = [conversionSymbol: string, volumeto: double ... 9 more fields]


lastException: Throwable = null


[conversionSymbol: string, volumeto: double ... 9 more fields]

In [217]:
// Saving data as Text

// don't forget to dele the previous folder 
// RDD does not allow us to overwrite: https://community.cloudera.com/t5/Support-Questions/Apache-SPARK-Overwrite-data-file/m-p/105253
val df_3 = df_2.withColumn("empty", typedLit(Seq(1)))
df_3.rdd.repartition(1).saveAsTextFile("data_cleaned")

println("All good")

All good


df_3 = [conversionSymbol: string, volumeto: double ... 10 more fields]


[conversionSymbol: string, volumeto: double ... 10 more fields]